In [11]:
!pip install python-dotenv



In [12]:
import pandas as pd
import sys  
import os
import nest_asyncio
import numpy as np
from telethon.sync import TelegramClient
import re
import emoji
import csv
from dotenv import load_dotenv


# Selecting 5 channels nand Scrape messages 

In [ ]:


# Allow nested asyncio loops (needed in notebooks like Jupyter)
nest_asyncio.apply()

# Load environment variables from .env file
load_dotenv('.env')

# Retrieve API credentials
API_ID = os.getenv('TG_API_ID')
API_HASH = os.getenv('TG_API_HASH')

if API_ID is None:
    raise ValueError("API_ID not found in environment variables")
else:
    API_ID = int(API_ID)

if API_HASH is None:
    raise ValueError("API_HASH not found in environment variables")

# Async function to fetch messages and media from a channel
async def fetch_channel_messages(client, channel_handle, csv_writer, download_folder):
    channel_entity = await client.get_entity(channel_handle)
    channel_name = channel_entity.title
    async for msg in client.iter_messages(channel_entity, limit=10000):
        media_file_path = None
        if msg.media and hasattr(msg.media, 'photo'):
            filename = f"{channel_handle}_{msg.id}.jpg"
            media_file_path = os.path.join(download_folder, filename)
            await client.download_media(msg.media, media_file_path)

        csv_writer.writerow([
            channel_name,
            channel_handle,
            msg.id,
            msg.message,
            msg.date,
            media_file_path
        ])

# Main async function to run the scraper
async def run_scraper():
    client = TelegramClient('session_scraper', API_ID, API_HASH)
    await client.start()

    # Define where to save data and images
    base_folder = r"C:\Users\HP\10 Acadamy PRojects\New folder (4)\amharic-ecommerce-scraper\data"
    media_folder = os.path.join(base_folder, 'downloaded_images')
    os.makedirs(media_folder, exist_ok=True)

    csv_path = os.path.join(base_folder, 'output_telegram_data.csv')

    with open(csv_path, mode='w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Channel Title', 'Channel Handle', 'Message ID', 'Content', 'Timestamp', 'Media File'])

  target_channels = [
            'ZemenExpress',
            'meneshayeofficial',
            'ethio_brand_collection',
            'Shewabrand',
            'qnashcom'
        ]

        for channel in target_channels:
            print(f"Scraping channel: {channel} ...")
            await fetch_channel_messages(client, channel, writer, media_folder)
            print(f"✅ Completed scraping {channel}")

    await client.disconnect()

# Run the scraper in a notebook or async environment
await run_scraper()


Scraping channel: ZemenExpress ...
✅ Completed scraping ZemenExpress
Scraping channel: meneshayeofficial ...
✅ Completed scraping meneshayeofficial
Scraping channel: ethio_brand_collection ...
✅ Completed scraping ethio_brand_collection
Scraping channel: Shewabrand ...
✅ Completed scraping Shewabrand
Scraping channel: qnashcom ...
✅ Completed scraping qnashcom
